# Reconstruction of a density matrix

In this tutorial, a walkthrough of how to reconstruct a density matrix via training a pair of modified *Restricted Boltzmann Machines* is presented


## The density matrix to be reconstructed
The density matrix that will be be reconstructed is the density matrix associated with the 2-qubit W state

\begin{equation}
            \vert\psi \rangle = \frac{1}{\sqrt{2}}\vert 01\rangle + \frac{1}{\sqrt{2}}\vert10\rangle
\end{equation}

so that

\begin{equation}
            \rho = \vert\psi\rangle\langle\psi\vert
\end{equation}

with global depolarization probability $p_{dep} = 0.5$ such that

\begin{equation}
            \rho_{new} = \left(1 - p_{dep}\right)\rho + \frac{p_{dep}}{2^N} I
\end{equation}

where $I$ is the identity matrix, representing the maximally mixed state.

The example dataset, `N2_W_state_data.txt`, is comprised of 900 $\sigma$ measurements, 100 in each of the $3^2$ permuations of two of the bases X, Y and Z. A corresponding file containing the bases for each data point in `N2_W_state_data.txt`, `N2_W_state_bases.txt`, is also required. The set of all 3^2 bases in which measurements are made is stored in `N2_IC_bases.txt`. Finally, the real and imaginary parts of the matrix are stored in `N2_W_state_target_real.txt` and `N2_W_state_target_imag.txt` respectively. As per convention, spins are represented in binary notation with zero and one denoting spin-up and spin-down, respectively.

## Using qucumber to reconstruct the density matrix

### Imports
To begin the tutorial, first import the required Python packages.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from qucumber.nn_states import DensityMatrix

from qucumber.callbacks import MetricEvaluator
import qucumber.utils.unitaries as unitaries
import qucumber.utils.cplx as cplx

import qucumber.utils.training_statistics as ts
import qucumber.utils.data as data

The Python class `DensityMatrix` contains the properties of an RBM needed to reconstruct the density matrix, as demonstrated in [this paper here].

To instantiate a `DensityMatrix` object, one needs to specify the number of visible, hidden and auxiliary units in the RBM. The number of visible units, `num_visible`, is given by the size of the physical system, i.e. the number of spins or qubits (2 in this case). On the other hand, the number of hidden units, `num_hidden`, can be varied to change the expressiveness of the neural network, and the number of auxiliary units, `num_aux`, can be varied depending on the extent of purification required of the system.

On top of needing the number of visible, hidden and auxiliary units, a `DensityMatrix` object requires the user to input a dictionary containing the unitary operators (2x2) that will be used to rotate the qubits in and out of the computational basis, Z, during the training process. The `unitaries` utility will take care of creating this dictionary.

The `MetricEvaluator` class and `training_statistics` utility are built-in amenities that will allow the user to evaluate the training in real time. 

Lastly, the `cplx` utility allows QuCumber to be able to handle complex numbers. Currently, PyTorch does not support complex numbers.


### Training
To evaluate the training in real time, the fidelity between the true wavefunction of the system and the wavefunction that QuCumber reconstructs, 

\begin{equation}
\operatorname{Tr }\sqrt{\sqrt{\rho_{RBM}}\rho\sqrt{\rho_{RBM}}}
\end{equation}
will be calculated along with the Kullback-Leibler (KL) divergence (the RBM's cost function). First, the training data and the true wavefunction of this system need to be loaded using the `data` utility.

[this paper here]: https://arxiv.org/pdf/1801.09684.pdf

In [2]:
train_path = "N2_W_state_data.txt"
train_bases_path = "N2_W_state_bases.txt"
matrix_path_real = "N2_W_state_target_real.txt"
matrix_path_imag = "N2_W_state_target_imag.txt"
bases_path = "N2_IC_bases.txt"


train_samples, train_bases, bases = data.load_data(
    train_path, tr_bases_path=train_bases_path, bases_path=bases_path
)

true_matrix_real = data.load_data(matrix_path_real)[0]
true_matrix_imag = data.load_data(matrix_path_imag)[0]

true_matrix = cplx.make_complex(true_matrix_real, true_matrix_imag)

The file `N2_all_bases.txt` contains every unique basis in the `N2_W_state_p0.50_100_samples_data.txt` file. Calculation of the full KL divergence in every basis requires the user to specify each unique basis.

As previously mentioned, a `DensityMatrix` object requires a dictionary that contains the unitary operators that will be used to rotate the qubits in and out of the computational basis, Z, during the training process. In the case of the provided dataset, the unitaries required are the well-known $H$, and $K$ gates. The dictionary needed can be created with the following command.

In [3]:
unitary_dict = unitaries.create_dict()
# unitary_dict = unitaries.create_dict(unitary_name=torch.tensor([[real part],
#                                                                 [imaginary part]],
#                                                                 dtype=torch.double)

If the user wishes to add their own unitary operators from their experiment to `unitary_dict`, uncomment the block above. When `unitaries.create_dict()` is called, it will contain the identity and the $H$ and $K$ gates by default under the keys "Z", "X" and "Y", respectively.

The number of visible units in the RBM is equal to the number of qubits. The number of hidden units will also be taken to be the number of visible units.

In [4]:
nv = train_samples.shape[-1]
nh = 1
na = 2

nn_state = DensityMatrix(
    num_visible=nv, num_hidden=nh, num_aux=na, unitary_dict=unitary_dict
)

The number of visible, hidden, and auxiliary units must now be specified. These are given by `nv`, `nh` and `na` respectively. The number of visible units is equal to the size of the system. The hidden and auxiliary units are hyperparameters that must be determiend by the user. With these, a `DensityMatrix` object can be instantiated.

Now the hyperparameters of the training process can be specified. 

1. `epochs`: the total number of training cycles that will be performed (default = 100)
2. `pos_batch_size`: the number of data points used in the positive phase of the gradient (default = 100)
3. `neg_batch_size`: the number of data points used in the negative phase of the gradient (default = `pos_batch_size`)
4. `k`: the number of contrastive divergence steps (default = 1)
5. `lr`: coefficient that scales the default value of the (non-constant) learning rate of the Adadelta algorithm (default = 1)

    **Note:** For more information on the hyperparameters above, it is strongly encouraged that the user to read through the brief, but thorough theory document on RBMs. One does not have to specify these hyperparameters, as their default values will be used without the user overwriting them. It is recommended to keep with the default values until the user has a stronger grasp on what these hyperparameters mean. The quality and the computational efficiency of the training will highly depend on the choice of hyperparameters. As such, playing around with the hyperparameters is almost always necessary. 

In [5]:
epochs = 250
pbs = 10  # pos_batch_size
nbs = 10  # neg_batch_size
lr = 5
k = 10

For evaluating the training in real time, the `MetricEvaluator` will be called to calculate the training evaluators every `period` epochs. The `MetricEvaluator` requires the following arguments.

1. `period`: the frequency of the training evaluators being calculated (e.g. `period=200` means that the `MetricEvaluator` will compute the desired metrics every 200 epochs)
2. A dictionary of functions you would like to reference to evaluate the training (arguments required for these functions are keyword arguments placed after the dictionary)

The following additional arguments are needed to calculate the fidelity and KL divergence in the `training_statistics` utility.

- `target_matrix` (the true density matrix of the system)
- `space` (the entire Hilbert space of the system)

The training evaluators can be printed out via the `verbose=True` statement.

Although the fidelity and KL divergence are excellent training evaluators, they are not practical to calculate in most cases; the user may not have access to the target wavefunction of the system, nor may generating the Hilbert space of the system be computationally feasible. However, evaluating the training in real time is extremely convenient. 

Any custom function that the user would like to use to evaluate the training can be given to the `MetricEvaluator`, thus avoiding having to calculate fidelity and/or KL divergence. As an example, a function to compute the partition function of the current density matrix is presented. Any custom function given to `MetricEvaluator` must take the neural-network state (in this case, the `Density` object) and keyword arguments. Although the given example requires the Hilbert space to be computed, the scope of the `MetricEvaluator`'s ability to be able to handle any function should still be evident.

In [6]:
def partition(nn_state, space, **kwargs):
    a_space = nn_state.generate_space(nn_state.num_aux)
    return nn_state.rbm_am.partition(space, a_space)

Now the Hilbert space of the system must be generated for the fidelity and KL divergence and the dictionary of functions the user would like to compute every `period` epochs must be given to the `MetricEvaluator`.

In [7]:
period = 1
space = nn_state.generate_hilbert_space(nv)
aux_space = nn_state.generate_hilbert_space(na)

callbacks = [
    MetricEvaluator(
        period,
        {
            "Density Matrix Fidelity": ts.density_matrix_fidelity,
            "Density Matrix KL": ts.density_matrix_KL,
            # "Partition Function": partition
        },
        target=true_matrix,
        bases=bases,
        verbose=True,
        v_space=space,
        a_space=aux_space,
    )
]

Now the training can begin. The `DensityMatrix` object has a function called `fit` which takes care of this.

In [ ]:
nn_state.fit(
    train_samples,
    train_bases,
    true_matrix,
    epochs=epochs,
    pos_batch_size=pbs,
    neg_batch_size=nbs,
    lr=lr,
    k=k,
    bases=bases,
    progbar="notebook",
    callbacks=callbacks,
    train_to_fid=0.999,
)

Epoch: 1	Density Matrix Fidelity = 0.641332	Density Matrix KL = 0.474613
Epoch: 2	Density Matrix Fidelity = 0.650911	Density Matrix KL = 0.444645
Epoch: 3	Density Matrix Fidelity = 0.678865	Density Matrix KL = 0.425416
Epoch: 4	Density Matrix Fidelity = 0.719939	Density Matrix KL = 0.377686
Epoch: 5	Density Matrix Fidelity = 0.796993	Density Matrix KL = 0.265148
Epoch: 6	Density Matrix Fidelity = 0.893291	Density Matrix KL = 0.090332
Epoch: 7	Density Matrix Fidelity = 0.909510	Density Matrix KL = 0.051538
Epoch: 8	Density Matrix Fidelity = 0.918888	Density Matrix KL = 0.034224
Epoch: 9	Density Matrix Fidelity = 0.929784	Density Matrix KL = 0.035206
Epoch: 10	Density Matrix Fidelity = 0.926544	Density Matrix KL = 0.037504
Epoch: 11	Density Matrix Fidelity = 0.888845	Density Matrix KL = 0.046477
Epoch: 12	Density Matrix Fidelity = 0.938697	Density Matrix KL = 0.031310
Epoch: 13	Density Matrix Fidelity = 0.940024	Density Matrix KL = 0.034907
Epoch: 14	Density Matrix Fidelity = 0.948373	De

In [ ]:
print(nn_state.rbm_am.weights_W)
print(nn_state.rbm_am.weights_U)
print(nn_state.rbm_am.visible_bias)
print(nn_state.rbm_am.hidden_bias)
print(nn_state.rbm_am.aux_bias)
print(nn_state.rbm_ph.weights_W)
print(nn_state.rbm_ph.weights_U)
print(nn_state.rbm_ph.visible_bias)
print(nn_state.rbm_ph.hidden_bias)
print(nn_state.rbm_ph.aux_bias)

All of these training evaluators can be accessed after the training has completed, as well. The code below shows this, along with plots of each training evaluator versus the training cycle number (epoch).

In [ ]:
fidelities = callbacks[0]["Density Matrix Fidelity"]
KLs = callbacks[0]["Density Matrix KL"]
epoch = np.arange(period, epochs + 1, period)

In [ ]:
# Some parameters to make the plots look nice

"""params = {
text.usetex: True,
font.family: serif,
legend.fontsize: 14,
figure.figsize: (10, 3),
axes.labelsize: 16,
xtick.labelsize: 14,
ytick.labelsize: 14,
lines.linewidth: 2,
lines.markeredgewidth: 0.8,
lines.markersize: 5,
lines.marker: o,
patch.edgecolor: black,
},
plt.rcParams.update(params),
plt.style.use(seaborn-deep)"""

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(14, 3))
ax = axs[0]
ax.plot(epoch, fidelities, "o", color="C0", markeredgecolor="black")
ax.set_ylabel(r"Fidelity")
ax.set_xlabel(r"Epoch")

ax = axs[1]
ax.plot(epoch, KLs, "o", color="C1", markeredgecolor="black")
ax.set_ylabel(r"KL Divergence")
ax.set_xlabel(r"Epoch")

# ax = axs[2]
# ax.plot(epoch, partition, "o", color="C2", markeredgecolor="black")
# ax.set_ylabel(r"$\vert\alpha\vert$")
# ax.set_xlabel(r"Epoch")

This saves the weights, visible biases and hidden biases as torch tensors with the following keys: "weights", "visible_bias", "hidden_bias".

In [ ]:
nn_state.save("saved_params_W_state.pt")